Antes de cualquier cosa, debes autentificarte a través del cmd, ejecutando:<br>
earthengine authenticate <br>
y antes deberías instalar pip install earthengine-api geemap geopandas pandas

In [1]:
import ee
import geopandas as gpd
import pandas as pd
from datetime import datetime
from shapely.geometry import mapping
import json
from tqdm import tqdm
import csv

Hay que ir a 
https://console.cloud.google.com/apis/dashboard?invt=AbuKWg&project=maximal-osprey-XXX
para habilitar la consulta.

In [2]:
# Inicializar Earth Engine
ee.Initialize(project='maximal-osprey-XXX')  # Reemplaza con tu ID real

In [3]:
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

In [4]:
parroquias = gdf[['ADM3_PCODE', 'geometry']]
#parroquias[parroquias['ADM3_PCODE'] == 'EC160156']
#parroquias = parroquias[parroquias['ADM3_PCODE'].isin(['EC050651', 'EC010951'])] #'EC050651',
parroquias = parroquias.iloc[775:]

In [5]:
# Función para convertir a EE FeatureCollection correctamente
def gdf_to_fc(tabla_geometria):
    features = []
    for _, row in tabla_geometria.iterrows():
        # Convierte la geometría a formato geojson (dict) con mapping()
        geojson_geom = mapping(row['geometry'])
        geom = ee.Geometry(geojson_geom)
        props = row.drop('geometry').to_dict()
        features.append(ee.Feature(geom, props))
    return ee.FeatureCollection(features)

# Función para calcular MNDWI (Green - SWIR1) / (Green + SWIR1)
def calcular_mndwi(img):
    mndwi = img.normalizedDifference(['B3', 'B11']).rename('MNDWI')  # Green, SWIR1
    return img.addBands(mndwi)

In [6]:
# Definir años a procesar
#years = [a for a in range(2015, 2025)]
years = [2022]

# Abrir el archivo en modo escritura (o append si ya existe)
with open('C:/Users/Michael/Documents/ml_zc/DatosObtenidos/mndwi2.csv', mode='w', newline='', encoding='utf-8') as file:
    # Crear el escritor CSV
    writer = csv.DictWriter(file, fieldnames=['ADM3_PCODE', 'Año', 'avg_mndwi'])
    
    # Escribir el encabezado (si es necesario)
    writer.writeheader()
    
    for num_fila in tqdm(range(0, parroquias.shape[0]), desc="Procesando parroquias", ncols=parroquias.shape[0]):
        
        # Obtener fila de la parroquia
        parroquia = parroquias.iloc[num_fila]
    
        # Convertir la fila (Serie) a un GeoDataFrame
        temp_gdf = gpd.GeoDataFrame([parroquia], geometry='geometry', crs=parroquias.crs)
    
        # Transformamos el objeto a formato compatible con Google Earth Engine
        polygon = gdf_to_fc(temp_gdf)

        # Extraemos el código de la parroquia para usarla luego
        name_parr = parroquia['ADM3_PCODE']

        # Mostrar parroquia en proceso
        print(name_parr)

        # Extraer VIIRS diciembre por parroquia:

        # Iterar sobre el periodo
        for year in years:
            start = ee.Date.fromYMD(year, 1, 1)  # Comienza desde el 1 de enero
            end = ee.Date.fromYMD(year, 12, 31)   # Termina el 31 de diciembre
            
            # Son datos 2015 - 2025.  Ver en: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2?hl=es-419
            image_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(polygon)\
                    .filterDate(start, end)\
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
            # Verificar cuántas imágenes fueron seleccionadas
            num_images = image_collection.size().getInfo()
            print(f"     Imágenes seleccionadas para {year}: {num_images}")

            if num_images > 0:
                # Aplicar la función para calcular el NDVI a todas las imágenes
                mndwi_collection = image_collection.map(calcular_mndwi)

                # Calcular la media de NDVI para todo el periodo
                mndwi_mean = mndwi_collection.mean()

                # Reducir el NDVI por el polígono (media dentro del polígono)
                mndwi_zonal = mndwi_mean.select('MNDWI').reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=polygon,
                    scale=10,
                    maxPixels=1e8
                )

                mndwi_zonal_info = mndwi_zonal.getInfo()
                print(f"     Resultado de MNDWI para {year}: {mndwi_zonal_info}")
                
                if mndwi_zonal_info and 'MNDWI' in mndwi_zonal_info:
                    writer.writerow({'ADM3_PCODE': name_parr, 'Año': year, 'avg_mndwi': mndwi_zonal_info['MNDWI']})
                else:
                    print(f"       >> No se obtuvieron valores de MNDWI para {year}")
            else:
                print(f"     >> No se encontraron imágenes para el año {year}")

Procesando parroquias:   0%|                                                                                                                                                                                                                       | 0/267 [00:00<?, ?it/s]

EC160157
     Imágenes seleccionadas para 2022: 4


Procesando parroquias:   0%|▊                                                                                                                                                                                                              | 1/267 [00:06<26:55,  6.07s/it]

     Resultado de MNDWI para 2022: {'MNDWI': -0.27996126059957943}
EC160158
     Imágenes seleccionadas para 2022: 16


Procesando parroquias:   1%|█▌                                                                                                                                                                                                           | 2/267 [00:43<1:48:02, 24.46s/it]

...

     Resultado de MNDWI para 2022: {'MNDWI': -0.08102864806684607}
EC240351
     Imágenes seleccionadas para 2022: 21


Procesando parroquias: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 266/267 [54:04<00:07,  7.26s/it]

     Resultado de MNDWI para 2022: {'MNDWI': -0.2683692418850825}
EC240352
     Imágenes seleccionadas para 2022: 21


Procesando parroquias: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 267/267 [54:11<00:00, 12.18s/it]

     Resultado de MNDWI para 2022: {'MNDWI': -0.14629306987400953}
